<a href="https://colab.research.google.com/github/unt-iialab/medical-concept-normalization/blob/master/medical-concept-normalization/fine-tuning-experiments/askpatient/pubmed_ask_01.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os

In [2]:
base_dir = 'medical-concept-normalization'

In [3]:
os.mkdir(base_dir)

In [4]:
os.mkdir('medical-concept-normalization/data_collection')

**Install fastai and clone the fastai for text classification**



In [5]:
!curl https://course-v3.fast.ai/setup/colab | bash

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0curl: (6) Could not resolve host: course-v3.fast.ai


In [6]:
!git clone https://github.com/wshuyi/demo-nlp-classification-fastai.git

Cloning into 'demo-nlp-classification-fastai'...
remote: Enumerating objects: 9, done.
remote: Total 9 (delta 0), reused 0 (delta 0), pack-reused 9
Unpacking objects: 100% (9/9), done.


In [7]:
from fastai import *
from fastai.text import *

**Set the path of the data collections**

In [8]:
path = Path('medical-concept-normalization/data_collection')

In [10]:
# training data set

train_csv = path/'AskAPatient.fold-1.train.csv'
train = pd.read_csv(train_csv, header=None)
#train.head()
#train.info()

In [11]:
# validating data set

valid = pd.read_csv(path/"AskAPatient.fold-1.validation.csv",header=None)
valid.head()

,0,1
0,0,fear
1,2,heels hurt
2,4,Discoloration of skin [blotchy] on the neck
3,6,diastolic dysfunction
4,8,arthritis


In [12]:
# testing data set

test = pd.read_csv(path/"AskAPatient.fold-1.test.csv",header=None)
test.head()

,0,1
0,2,heel pain
1,2,right heel became painful
2,8,Arthritis
3,8,arthristis
4,8,arthritis


In [13]:
# Creating DataBunch for language modelling

data_lm = TextLMDataBunch.from_csv(path, 'pubmed_ask.csv')

In [14]:
# Creating DataBunch for classification

# data_clas = TextClasDataBunch.from_df(path, train, valid, test, vocab=data_lm.train_ds.vocab, bs=128)

In [15]:
data_clas = TextClasDataBunch.from_df(path, train, valid, vocab=data_lm.train_ds.vocab, bs=128)

In [16]:
data_clas.add_test(test,label=0)


In [17]:
data_lm.save()
data_clas.save()

In [18]:
data_lm.train_ds.vocab.itos

['xxunk',
 'xxpad',
 'xxbos',
 'xxeos',
 'xxfld',
 'xxmaj',
 'xxup',
 'xxrep',
 'xxwrep',
 'pain',
 'of',
 'the',
 'in',
 'lipitor',
 'and',
 'to',
 'muscle',
 'a',
 'was',
 'diclofenac',
 'with',
 'were',
 'severe',
 'for',
 'myalgia',
 'lower',
 'my',
 'fatigue',
 'loss',
 'sodium',
 'limb',
 'cramp',
 'weakness',
 'on',
 'by',
 'stomach',
 'depression',
 '-',
 'back',
 'is',
 'at',
 'cramps',
 'patients',
 'that',
 'joint',
 'as',
 'gas',
 'upper',
 'or',
 'memory',
 'drug',
 'leg',
 'mg',
 'legs',
 'pains',
 'knee',
 'from',
 'headache',
 'group',
 'unable',
 'shoulder',
 'not',
 'arthralgia',
 'study',
 'gastrointestinal',
 'foot',
 'neck',
 'all',
 'arthritis',
 'excessive',
 'treatment',
 'this',
 'aches',
 'after',
 'hip',
 'voltaren',
 'disease',
 'p',
 'numbness',
 'insomnia',
 'arthrotec',
 'release',
 'nausea',
 'increased',
 'both',
 '/',
 'bleeding',
 'an',
 'swelling',
 'be',
 'asthenia',
 'walk',
 'abdominal',
 'like',
 'skin',
 'dizziness',
 'tired',
 'heart',
 'feelin

In [19]:
model_path = path/'models'
model_path.mkdir(exist_ok=True)
url = 'http://files.fast.ai/models/wt103_v1/'
download_url(f'{url}lstm_wt103.pth', model_path/'lstm_wt103.pth')
download_url(f'{url}itos_wt103.pkl', model_path/'itos_wt103.pkl')

In [20]:
learn = language_model_learner(data_lm, AWD_LSTM, drop_mult=0.5)

In [21]:
learn.fit_one_cycle(1, 1e-2)

epoch,train_loss,valid_loss,accuracy,time
0,3.792674,3.363943,0.460527,28:39


In [22]:
# learn.fit_one_cycle(1, 1e-2)

In [23]:
# learn.fit_one_cycle(1, 1e-3)

In [24]:
learn.fit_one_cycle(1, 1e-3)

epoch,train_loss,valid_loss,accuracy,time
0,3.419755,3.305378,0.465796,28:31


In [25]:
learn.save_encoder('pubmed_ask_fold1_first')

In [26]:
learn = text_classifier_learner(data_clas,AWD_LSTM, drop_mult=0.5)
learn.load_encoder('pubmed_ask_fold1_first')
learn.fit_one_cycle(1, 1e-2)

epoch,train_loss,valid_loss,accuracy,time
0,3.988538,4.085874,0.296610,01:11


In [27]:
learn.freeze_to(-2)
learn.fit_one_cycle(1, slice(5e-3/2., 5e-3))

epoch,train_loss,valid_loss,accuracy,time
0,2.918314,3.331257,0.406780,01:18


In [28]:
learn.unfreeze()
learn.fit_one_cycle(1, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,2.313232,2.882601,0.491525,03:12


In [29]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,2.109271,2.441298,0.582324,03:23
1,1.694445,2.293300,0.587167,03:28


In [30]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,1.547839,2.087015,0.629540,03:11
1,1.336053,1.974126,0.644068,03:28


In [31]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,1.253805,1.852033,0.658596,03:18
1,1.034443,1.783723,0.676755,03:07


In [32]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,1.042234,1.664504,0.687651,03:17
1,0.883389,1.594533,0.709443,03:20


In [33]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.872841,1.543494,0.713075,03:20
1,0.733417,1.491633,0.720339,03:21


In [34]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.653298,1.514907,0.717918,03:11
1,0.644361,1.457106,0.746973,03:25
2,0.575989,1.401924,0.757869,03:26
3,0.500190,1.362006,0.754237,03:10


In [35]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.511425,1.412743,0.743341,03:28
1,0.538767,1.355369,0.757869,03:18
2,0.442469,1.353999,0.767554,03:31
3,0.365727,1.348150,0.767554,03:26


In [36]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.369913,1.469133,0.761501,03:20
1,0.431654,1.305091,0.772397,03:19
2,0.363883,1.273125,0.773608,03:18
3,0.283950,1.273995,0.780872,03:31


In [37]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.291295,1.269362,0.780872,03:12
1,0.368972,1.302038,0.782082,03:26
2,0.311643,1.306606,0.782082,03:13
3,0.261665,1.271550,0.785714,03:24


In [38]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.270483,1.301264,0.780872,03:10
1,0.306187,1.361474,0.784504,03:06
2,0.254360,1.312364,0.796610,03:24
3,0.227568,1.302461,0.795399,03:13


In [39]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.243514,1.297367,0.790557,03:15
1,0.278538,1.403838,0.769976,03:28
2,0.227178,1.381421,0.784504,03:10
3,0.205737,1.323540,0.792978,03:27


In [40]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.200482,1.339136,0.802663,03:32
1,0.255295,1.321740,0.802663,03:17
2,0.221992,1.397917,0.805085,03:27
3,0.191084,1.399530,0.802663,03:31


In [41]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.207797,1.435889,0.792978,03:30
1,0.193162,1.373461,0.803874,03:14


In [42]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.204230,1.445872,0.794189,03:15
1,0.194671,1.415408,0.795399,03:13


In [43]:
learn.fit_one_cycle(1, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.209764,1.361379,0.803874,03:32


In [44]:
# save the best model

learn.save_encoder('pubmed_ask_mixed_fold1')

# Part three: Predict on the test dataset

In [45]:
# use the test data for prediction

preds, y = learn.get_preds(DatasetType.Test)

In [46]:
predictions, *_ = learn.get_preds(DatasetType.Test)
labels = np.argmax(predictions, 1)
predict_list = labels.tolist()
standard_list = test[0].tolist()

print(len(predict_list))
print(len(standard_list))

print(predict_list)
print(standard_list)

867
867
[2, 2, 8, 1026, 8, 8, 1026, 16, 20, 21, 693, 246, 27, 31, 36, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 42, 42, 44, 813, 49, 49, 49, 49, 49, 49, 49, 49, 53, 53, 53, 226, 60, 61, 62, 626, 70, 776, 71, 122, 79, 79, 79, 79, 79, 79, 79, 79, 79, 79, 79, 79, 79, 79, 79, 79, 79, 79, 79, 79, 79, 79, 79, 79, 79, 79, 79, 79, 108, 108, 108, 108, 108, 108, 79, 111, 135, 80, 17, 80, 80, 80, 80, 491, 109, 88, 89, 240, 92, 92, 92, 94, 94, 94, 94, 94, 830, 94, 94, 94, 94, 96, 847, 908, 234, 100, 31, 100, 100, 766, 100, 100, 572, 301, 100, 100, 58, 103, 964, 105, 107, 107, 108, 108, 111, 111, 462, 111, 111, 111, 111, 111, 111, 111, 114, 117, 395, 125, 125, 125, 125, 125, 125, 125, 125, 125, 125, 125, 125, 125, 125, 802, 37, 624, 847, 693, 135, 693, 139, 139, 57, 142, 576, 864, 146, 158, 166, 343, 166, 169, 171, 172, 172, 172, 172, 172, 173, 173, 173, 106, 180, 182, 182, 794, 182, 182, 182, 182, 182, 182, 182, 182, 183, 183, 183, 183, 183, 183, 183, 184, 187, 187, 187, 187, 189, 189, 190, 190,

In [47]:
correct_num = 0

for i in range(0, len(standard_list)):
  if predict_list[i] == standard_list[i]:
    correct_num = correct_num+1
  else:
    pass
print(correct_num)

predict_accuracy = correct_num/len(predict_list)
print(predict_accuracy)

670
0.7727797001153403
